# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [29]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf, count, when, isnull, collect_list
from pyspark.sql.types import IntegerType, BooleanType, FloatType
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler

In [2]:
# create a Spark session
spark = SparkSession.builder.appName("Sparkify").getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
data_path = 'mini_sparkify_event_data.json'
df = spark.read.json(data_path)
# See the frame schema
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [4]:
# Total number of records in our dataset
df.count()

286500

In [5]:
df.select([count(when(isnull(x), x)).alias(x) for x in df.columns]).show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId| song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
| 58392|   0|     8346|  8346|            0|    8346| 58392|    0|    8346|     0|   0|        8346|        0|58392|     0|  0|     8346|     0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+



In [6]:
# Dropping the rows with NA userId and sessionId
df = df.dropna(how='any', subset=["userId", "sessionId"])
df.count()

286500

In [7]:
df.select('auth').distinct().show()

+----------+
|      auth|
+----------+
|Logged Out|
| Cancelled|
|     Guest|
| Logged In|
+----------+



In [8]:
df.select('level').distinct().show()

+-----+
|level|
+-----+
| free|
| paid|
+-----+



In [9]:
df.select('location').distinct().show()

+--------------------+
|            location|
+--------------------+
|     Gainesville, FL|
|Atlantic City-Ham...|
|Deltona-Daytona B...|
|San Diego-Carlsba...|
|Cleveland-Elyria, OH|
|Kingsport-Bristol...|
|New Haven-Milford...|
|Birmingham-Hoover...|
|  Corpus Christi, TX|
|         Dubuque, IA|
|Las Vegas-Henders...|
|Indianapolis-Carm...|
|Seattle-Tacoma-Be...|
|          Albany, OR|
|   Winston-Salem, NC|
|     Bakersfield, CA|
|Los Angeles-Long ...|
|Minneapolis-St. P...|
|San Francisco-Oak...|
|Phoenix-Mesa-Scot...|
+--------------------+
only showing top 20 rows



In [10]:
df.select('page').distinct().show(10)

+--------------------+
|                page|
+--------------------+
|              Cancel|
|    Submit Downgrade|
|         Thumbs Down|
|                Home|
|           Downgrade|
|         Roll Advert|
|              Logout|
|       Save Settings|
|Cancellation Conf...|
|               About|
+--------------------+
only showing top 10 rows



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [11]:
clean_data = df.select('artist','auth','firstName','gender','lastName','length','level','location','page','song','ts','userId')
clean_data.where((col("page") == "Cancellation Confirmation")).select("userId").distinct().count()

52

In [12]:
clean_data.filter(clean_data.auth=="Cancelled").show()

+------+---------+---------+------+---------+------+-----+--------------------+--------------------+----+-------------+------+
|artist|     auth|firstName|gender| lastName|length|level|            location|                page|song|           ts|userId|
+------+---------+---------+------+---------+------+-----+--------------------+--------------------+----+-------------+------+
|  null|Cancelled|   Adriel|     M|  Mendoza|  null| paid|  Kansas City, MO-KS|Cancellation Conf...|null|1538943990000|    18|
|  null|Cancelled|    Diego|     M|    Mckee|  null| paid|Phoenix-Mesa-Scot...|Cancellation Conf...|null|1539033046000|    32|
|  null|Cancelled|    Mason|     M|     Hart|  null| free|  Corpus Christi, TX|Cancellation Conf...|null|1539318918000|   125|
|  null|Cancelled|Alexander|     M|   Garcia|  null| paid|Indianapolis-Carm...|Cancellation Conf...|null|1539375441000|   105|
|  null|Cancelled|    Kayla|     F|  Johnson|  null| paid|Philadelphia-Camd...|Cancellation Conf...|null|153946

In [13]:
# Filtering churned users
user_churned = clean_data.filter(clean_data.auth=="Cancelled")

In [14]:
# Grouping by userId
churn_df = clean_data.groupby('userId').agg(collect_list('auth').alias("auths"))

# Filtering churned user with lambda functions
churned = udf(lambda x: 'Cancelled' in x)
churn_df = churn_df.withColumn("Churned", churned(churn_df.auths))

# Dropping alias
churn_df = churn_df.drop('auths')

# Joining churned data with clean data
label = churn_df.join(clean_data,'userId')

label.show()

+------+-------+--------------------+---------+---------+------+---------+---------+-----+--------------------+-----------+--------------------+-------------+
|userId|Churned|              artist|     auth|firstName|gender| lastName|   length|level|            location|       page|                song|           ts|
+------+-------+--------------------+---------+---------+------+---------+---------+-----+--------------------+-----------+--------------------+-------------+
|100010|  false|Sleeping With Sirens|Logged In| Darianna|     F|Carpenter|202.97098| free|Bridgeport-Stamfo...|   NextSong|Captain Tyin Knot...|1539003534000|
|100010|  false|Francesca Battist...|Logged In| Darianna|     F|Carpenter|196.54485| free|Bridgeport-Stamfo...|   NextSong|Beautiful_ Beauti...|1539003736000|
|100010|  false|              Brutha|Logged In| Darianna|     F|Carpenter|263.13098| free|Bridgeport-Stamfo...|   NextSong|          She's Gone|1539003932000|
|100010|  false|                null|Logged In

In [15]:
# Churned user table
churn_df.show()

+------+-------+
|userId|Churned|
+------+-------+
|100010|  false|
|200002|  false|
|   125|   true|
|   124|  false|
|    51|   true|
|     7|  false|
|    15|  false|
|    54|   true|
|   155|  false|
|100014|   true|
|   132|  false|
|   154|  false|
|   101|   true|
|    11|  false|
|   138|  false|
|300017|  false|
|100021|   true|
|    29|   true|
|    69|  false|
|   112|  false|
+------+-------+
only showing top 20 rows



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [16]:
# Creating columns for like and dislike of a song
th_up = label.where(label.page=='Thumbs Up').groupby("userId").agg(count(col('page')).alias('th_up')).orderBy('userId')
th_down = label.where(label.page=='Thumbs Down').groupby("userId").agg(count(col('page')).alias('th_down')).orderBy('userId')
like_dislike = th_up.join(th_down,'userId')
like_dislike.show()

+------+-----+-------+
|userId|th_up|th_down|
+------+-----+-------+
|100010|   17|      5|
|200002|   21|      6|
|   124|  171|     41|
|    51|  100|     21|
|     7|    7|      1|
|    15|   81|     14|
|    54|  163|     29|
|   155|   58|      3|
|100014|   17|      3|
|   132|   96|     17|
|   101|   86|     16|
|    11|   40|      9|
|   138|   95|     24|
|300017|  303|     28|
|100021|   11|      5|
|    29|  154|     22|
|    69|   72|      9|
|   112|    9|      3|
|    42|  166|     25|
|    73|   14|      7|
+------+-----+-------+
only showing top 20 rows



In [17]:
# Creating column for number of songs played by user
num_songs = label.where(col('song')!='null').groupby("userId").agg(count(col('song')).alias('num_songs')).orderBy('userId')

# Creating column for number of days a user has spent in the app
duration = label.groupby('userId').agg(((max(col('ts')) - min(col('ts')))/86400000).alias("duration"))

In [18]:
num_songs.show()

+------+---------+
|userId|num_songs|
+------+---------+
|    10|      673|
|   100|     2682|
|100001|      133|
|100002|      195|
|100003|       51|
|100004|      942|
|100005|      154|
|100006|       26|
|100007|      423|
|100008|      772|
|100009|      518|
|100010|      275|
|100011|       11|
|100012|      476|
|100013|     1131|
|100014|      257|
|100015|      800|
|100016|      530|
|100017|       52|
|100018|     1002|
+------+---------+
only showing top 20 rows



In [19]:
duration.show()

+------+-------------------+
|userId|           duration|
+------+-------------------+
|100010|  44.21780092592593|
|200002| 45.496805555555554|
|   125|0.02053240740740741|
|   124| 59.996944444444445|
|    51| 15.779398148148148|
|     7| 50.784050925925925|
|    15|  54.77318287037037|
|    54|  42.79719907407407|
|   155|  25.82783564814815|
|100014| 41.244363425925926|
|   132|  50.49740740740741|
|   154| 24.986458333333335|
|   101| 15.861481481481482|
|    11| 53.241585648148146|
|   138|  56.07674768518518|
|300017|  59.11390046296296|
|100021| 45.457256944444445|
|    29|  43.32092592592593|
|    69|  50.98648148148148|
|   112|  56.87869212962963|
+------+-------------------+
only showing top 20 rows



In [20]:
# Joining all the features
final_features = churn_df.join(like_dislike,'userId')
final_features = final_features.join(num_songs,'userId')
final_features = final_features.join(duration,'userId')
final_features.show()

+------+-------+-----+-------+---------+------------------+
|userId|Churned|th_up|th_down|num_songs|          duration|
+------+-------+-----+-------+---------+------------------+
|100010|  false|   17|      5|      275| 44.21780092592593|
|200002|  false|   21|      6|      387|45.496805555555554|
|   124|  false|  171|     41|     4079|59.996944444444445|
|    51|   true|  100|     21|     2111|15.779398148148148|
|     7|  false|    7|      1|      150|50.784050925925925|
|    15|  false|   81|     14|     1914| 54.77318287037037|
|    54|   true|  163|     29|     2841| 42.79719907407407|
|   155|  false|   58|      3|      820| 25.82783564814815|
|100014|   true|   17|      3|      257|41.244363425925926|
|   132|  false|   96|     17|     1928| 50.49740740740741|
|   101|   true|   86|     16|     1797|15.861481481481482|
|    11|  false|   40|      9|      647|53.241585648148146|
|   138|  false|   95|     24|     2070| 56.07674768518518|
|300017|  false|  303|     28|     3632|

To see if we got any null values

In [21]:
final_features.select([count(when(isnull(x), x)).alias(x) for x in ["userId", "Churned", "th_up", "th_down", "duration"]]).show()

+------+-------+-----+-------+--------+
|userId|Churned|th_up|th_down|duration|
+------+-------+-----+-------+--------+
|     0|      0|    0|      0|       0|
+------+-------+-----+-------+--------+



In [22]:
# Adding more features
up_song = udf(lambda Up, songs: float(Up)/float(songs), FloatType())
down_song = udf(lambda Down, songs: float(Down)/float(songs), FloatType())
song_hour = udf(lambda Songs, Days: float(Songs)/float((Days*24)), FloatType())

In [23]:
# Joining newly added features with final_features
final_features = final_features.withColumn("up_song", up_song(final_features.th_up, final_features.num_songs))
final_features = final_features.withColumn("down_song", down_song(final_features.th_down, final_features.num_songs))
final_features = final_features.withColumn("song_hour", song_hour(final_features.num_songs, final_features.duration))
final_features.show()

+------+-------+-----+-------+---------+------------------+-----------+------------+-----------+
|userId|Churned|th_up|th_down|num_songs|          duration|    up_song|   down_song|  song_hour|
+------+-------+-----+-------+---------+------------------+-----------+------------+-----------+
|100010|  false|   17|      5|      275| 44.21780092592593|0.061818182| 0.018181818| 0.25913393|
|200002|  false|   21|      6|      387|45.496805555555554|0.054263566| 0.015503876| 0.35442048|
|   124|  false|  171|     41|     4079|59.996944444444445| 0.04192204| 0.010051483|  2.8327832|
|    51|   true|  100|     21|     2111|15.779398148148148|0.047370914| 0.009947892|  5.5742517|
|     7|  false|    7|      1|      150|50.784050925925925|0.046666667| 0.006666667|0.123070136|
|    15|  false|   81|     14|     1914| 54.77318287037037| 0.04231975|0.0073145246|  1.4560045|
|    54|   true|  163|     29|     2841| 42.79719907407407|0.057374164| 0.010207674|   2.765952|
|   155|  false|   58|      3|

### Scaling the features

In [27]:
# Reference: Official Apache Spark Documentation (https://spark.apache.org/docs/latest/ml-features.html)
assembler = VectorAssembler(
    inputCols=["num_songs", "up_song", "down_song", "duration", "song_hour"],
    outputCol="vector_features")
final_features = assembler.transform(final_features)

In [31]:
# Reference: Official Apache Spark Documentation (https://spark.apache.org/docs/latest/ml-features.html)
scaler = MinMaxScaler(inputCol="vector_features", outputCol="scaledFeatures")
# Compute summary statistics and generate MinMaxScalerModel
scalerModel = scaler.fit(final_features)

# rescale each feature to range [min, max].
final_features = scalerModel.transform(final_features)
print("Features scaled to range: [%f, %f]" % (scaler.getMin(), scaler.getMax()))
final_features.select("vector_features", "scaledFeatures").show()

Features scaled to range: [0.000000, 1.000000]
+--------------------+--------------------+
|     vector_features|      scaledFeatures|
+--------------------+--------------------+
|[275.0,0.06181818...|[0.03121865596790...|
|[387.0,0.05426356...|[0.04526078234704...|
|[4079.0,0.0419220...|[0.50814944834503...|
|[2111.0,0.0473709...|[0.26140922768304...|
|[150.0,0.04666666...|[0.01554663991975...|
|[1914.0,0.0423197...|[0.23671013039117...|
|[2841.0,0.0573741...|[0.35293380140421...|
|[820.0,0.07073170...|[0.09954864593781...|
|[257.0,0.06614785...|[0.02896188565697...|
|[1928.0,0.0497925...|[0.23846539618856...|
|[1797.0,0.0478575...|[0.22204112337011...|
|[647.0,0.06182380...|[0.07785857572718...|
|[2070.0,0.0458937...|[0.25626880641925...|
|[3632.0,0.0834251...|[0.45210631895687...|
|[230.0,0.04782608...|[0.02557673019057...|
|[3028.0,0.0508586...|[0.37637913741223...|
|[1125.0,0.0640000...|[0.13778836509528...|
|[215.0,0.04186046...|[0.02369608826479...|
|[3573.0,0.0464595...|[0.4447

In [34]:
int_conversion = udf(lambda a: 1 if a=="true" else 0, IntegerType())
final_features = final_features.withColumn('label', int_conversion(final_features.Churned))

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.